# Création des fichiers

## Setup

### Import des fichiers

In [40]:
#Temps et fichiers
import os
import warnings
import time
from datetime import timedelta

#Manipulation de données
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
from functools import partial


#Modélisation
from sklearn.datasets import fetch_openml
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import PoissonRegressor, GammaRegressor
from sklearn.linear_model import TweedieRegressor
from sklearn.metrics import mean_tweedie_deviance
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer

from sklearn.metrics import mean_absolute_error, mean_squared_error, auc

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import LinearSVC
from sklearn.model_selection import RandomizedSearchCV# the keys can be accessed with final_pipeline.get_params().keys()
from sklearn.linear_model import LogisticRegression

from xgboost import XGBClassifier

#Text
import re

#Evaluation
from sklearn.metrics import f1_score, confusion_matrix


#Visualisation
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px


#Tracking d'expérience
import mlflow
import mlflow.sklearn

### Utilisation du code du projet packagé

In [41]:
#Cette cellule permet d'appeler la version packagée du projet et d'en assurer le reload avant appel des fonctions
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Configuration de l'experiment MLFlow

In [42]:
mlflow.tracking.get_tracking_uri()

'/mnt/experiments'

## Chargement des données brutes

In [43]:
data_folder = os.path.join('/mnt', 'data', 'raw')
all_raw_files = [os.path.join(data_folder, fname)
                    for fname in os.listdir(data_folder)]
all_raw_files

['/mnt/data/raw/Data',
 '/mnt/data/raw/Data.zip',
 '/mnt/data/raw/expo_test.csv',
 '/mnt/data/raw/expo_train.csv',
 '/mnt/data/raw/illustration',
 '/mnt/data/raw/primes2019.csv',
 '/mnt/data/raw/sin_train.csv']

In [103]:
random_state=42

In [104]:
expo_train = pd.read_csv('/mnt/data/raw/expo_train.csv', encoding='utf8', sep=',' )

In [105]:
expo_train.head()

,Unnamed: 0,EXPO,FORMULE,TYPE_RESIDENCE,TYPE_HABITATION,NB_PIECES,SITUATION_JURIDIQUE,NIVEAU_JURIDIQUE,VALEUR_DES_BIENS,OBJETS_DE_VALEUR,ZONIER,NBSIN_TYPE1_AN1,NBSIN_TYPE1_AN2,NBSIN_TYPE1_AN3,NBSIN_TYPE2_AN1,NBSIN_TYPE2_AN2,NBSIN_TYPE2_AN3,id,ANNEE
0,384538,1.000000,MEDIUM,PRINCIPALE,APPARTEMENT,1.0,PROPRIO,JUR1,3500.0,NIVEAU_1,B40,0,1,0,0,0.0,0,5,2017
1,441079,0.824657,CONFORT,PRINCIPALE,MAISON,NaN,PROPRIO,JUR1,0.0,NIVEAU_1,A11,0,1,0,0,0.0,0,9,2018
2,119668,1.000000,ESSENTIEL,PRINCIPALE,APPARTEMENT,3.0,LOCATAIRE,JUR1,35000.0,NIVEAU_1,B32,0,1,0,0,0.0,0,11,2017
3,5124,1.000000,ESSENTIEL,SECONDAIRE,MAISON,2.0,LOCATAIRE,JUR1,9000.0,NIVEAU_1,C24,0,1,0,0,0.0,0,13,2018
4,130065,1.000000,ESSENTIEL,PRINCIPALE,MAISON,1.0,LOCATAIRE,JUR1,20000.0,NIVEAU_1,C9,0,1,0,0,0.0,0,14,2017


In [106]:
expo_test = pd.read_csv('/mnt/data/raw/expo_test.csv', encoding='utf8', sep=',' )
expo_test.head()

,Unnamed: 0,EXPO,FORMULE,TYPE_RESIDENCE,TYPE_HABITATION,NB_PIECES,SITUATION_JURIDIQUE,NIVEAU_JURIDIQUE,VALEUR_DES_BIENS,OBJETS_DE_VALEUR,ZONIER,NBSIN_TYPE1_AN1,NBSIN_TYPE1_AN2,NBSIN_TYPE1_AN3,NBSIN_TYPE2_AN1,NBSIN_TYPE2_AN2,NBSIN_TYPE2_AN3,id,ANNEE
0,141755,1.000000,CONFORT,PRINCIPALE,APPARTEMENT,4.0,LOCATAIRE,JUR1,35000.0,NIVEAU_1,B43,0,1,0,0,NaN,0,2,2019
1,81642,0.468493,CONFORT,SECONDAIRE,MAISON,1.0,PROPRIO,JUR1,0.0,NIVEAU_1,C22,0,1,0,0,0.0,0,10,2019
2,60207,0.852055,ESSENTIEL,PRINCIPALE,APPARTEMENT,0.0,PROPRIO,JUR1,0.0,NIVEAU_1,B32,0,1,0,0,0.0,0,18,2019
3,164160,1.000000,CONFORT,PRINCIPALE,APPARTEMENT,3.0,LOCATAIRE,JUR1,50000.0,NIVEAU_1,C6,0,1,1,0,0.0,0,19,2019
4,33416,0.079452,CONFORT,SECONDAIRE,MAISON,1.0,PROPRIO,JUR1,9000.0,NIVEAU_1,C6,0,1,0,0,0.0,0,39,2019


In [107]:
sin_train = pd.read_csv('/mnt/data/raw/sin_train.csv', encoding='utf8', sep=';', decimal=',' )
sin_train.head()

,id,NB,COUT,ANNEE
0,15,1,521.43,2017
1,277,1,3000.19,2016
2,643,1,26.33,2018
3,668,1,462.36,2017
4,730,1,640.88,2018


In [108]:
primes2019 = pd.read_csv('/mnt/data/raw/primes2019.csv', encoding='utf8', sep=';', decimal=',' )
primes2019.head()

,id,prime
0,1000001,81.878323
1,1000002,92.986674
2,1000003,34.635234
3,1000004,70.135502


### Problème non résolu

La lecture du fichier dans un DataFrame renvoie un format `objet` pour les champs texte.

Ca nous a posé des problèmes dans les pipelines par la suite

On n'a pas été capable de les retransformer en de vrais champs texte

In [120]:
expo_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155651 entries, 0 to 155650
Data columns (total 20 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   EXPO                    155651 non-null  float64
 1   FORMULE                 155651 non-null  object 
 2   TYPE_RESIDENCE          155651 non-null  object 
 3   TYPE_HABITATION         155651 non-null  object 
 4   NB_PIECES               146301 non-null  float64
 5   SITUATION_JURIDIQUE     155651 non-null  object 
 6   NIVEAU_JURIDIQUE        155651 non-null  object 
 7   VALEUR_DES_BIENS        155651 non-null  float64
 8   OBJETS_DE_VALEUR        155651 non-null  object 
 9   ZONIER                  155651 non-null  object 
 10  NBSIN_TYPE1_AN1         155651 non-null  int64  
 11  NBSIN_TYPE1_AN3         155651 non-null  int64  
 12  NBSIN_TYPE2_AN1         155651 non-null  int64  
 13  NBSIN_TYPE2_AN2         138509 non-null  float64
 14  NBSIN_TYPE2_AN3     

In [130]:
df=expo_train
print(df.info())
for col in list(df.select_dtypes('object').columns):
    df[col]=str(df[col])
#df[list(df.select_dtypes('object').columns)]=pd.Series()
print(df[list(df.select_dtypes('object').columns)].info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155651 entries, 0 to 155650
Data columns (total 20 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   EXPO                    155651 non-null  float64
 1   FORMULE                 155651 non-null  object 
 2   TYPE_RESIDENCE          155651 non-null  object 
 3   TYPE_HABITATION         155651 non-null  object 
 4   NB_PIECES               146301 non-null  float64
 5   SITUATION_JURIDIQUE     155651 non-null  object 
 6   NIVEAU_JURIDIQUE        155651 non-null  object 
 7   VALEUR_DES_BIENS        155651 non-null  float64
 8   OBJETS_DE_VALEUR        155651 non-null  object 
 9   ZONIER                  155651 non-null  object 
 10  NBSIN_TYPE1_AN1         155651 non-null  int64  
 11  NBSIN_TYPE1_AN3         155651 non-null  int64  
 12  NBSIN_TYPE2_AN1         155651 non-null  int64  
 13  NBSIN_TYPE2_AN2         138509 non-null  float64
 14  NBSIN_TYPE2_AN3     

## Premier EDA rapide

On commence par créer un profile report pour se donner une idée

In [50]:
pr = ProfileReport(expo_train).to_file(output_file='expo_train.html')

Summarize dataset:   0%|          | 0/32 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/scipy/stats/stats.py:4594: RuntimeWarning: overflow encountered in long_scalars
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [51]:
pr = ProfileReport(expo_test).to_file(output_file='expo_test.html')

Summarize dataset:   0%|          | 0/32 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [52]:
pr = ProfileReport(sin_train).to_file(output_file='sin_train.html')

Summarize dataset:   0%|          | 0/17 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Insight

On voit plusieurs pb :
- une première colonne **unnamed** dans les expo_train et expo_test
- expo_train fait état de **sinistres de types 1 et 2**, alors que cette info n'est pas dans sinistres

**Question 1 :** On vérifie si un Id correspond bien à une police x une année d'observation

In [53]:
expo_train[['id', 'ANNEE']].groupby('id').agg('count').describe()

,ANNEE
count,155651.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


**=>** c'est bien le cas

**A FAIRE :** du coup on peut calculer le nombre de sinistres antérieurs

**Question 2 :** est-ce qu'on retrouve dans expo_train les sinistres ? 

## Création des données processées

Le fait de créer des fonctions permet de s'assurer que les mêmes traitements soient bien appliqués

In [109]:
def preprocess(df):
    #On vire la colonne unnamed :0 : André => ancien numéro de ligne
    #On vire `NBSIN_TYPE1_AN2` => vu avec André : pb dans la variable qui est >=1
    df=df.drop(['Unnamed: 0','NBSIN_TYPE1_AN2'], axis=1)
    df['ZONIER_2']=df['ZONIER'].astype(str).str[0]
    df['NBSIN_TYPE1_AN1_RECODE']= df['NBSIN_TYPE1_AN1'].apply(lambda x : 1 if x>1 else 0)
    df['NBSIN_TYPE1_AN3_RECODE']= df['NBSIN_TYPE1_AN3'].apply(lambda x : 1 if x>1 else 0)
    return df

In [110]:
expo_train=preprocess(expo_train)
expo_test=preprocess(expo_test)

In [111]:
def cree_df_merged(df_exp, df_sin):
    df_merged=pd.merge(df_exp, df_sin, on=['id', 'ANNEE'], how='left' )
    df_merged[['NB', 'COUT']]=df_merged[['NB', 'COUT']].fillna(0)
    df_merged['Isin']=df_merged['NB'].apply(lambda x : min(1, x))
    #on extraie la première lettre
    return df_merged

In [112]:
df_merged=cree_df_merged(expo_train, sin_train)

In [113]:
df_merged

,EXPO,FORMULE,TYPE_RESIDENCE,TYPE_HABITATION,NB_PIECES,SITUATION_JURIDIQUE,NIVEAU_JURIDIQUE,VALEUR_DES_BIENS,OBJETS_DE_VALEUR,ZONIER,...,NBSIN_TYPE2_AN2,NBSIN_TYPE2_AN3,id,ANNEE,ZONIER_2,NBSIN_TYPE1_AN1_RECODE,NBSIN_TYPE1_AN3_RECODE,NB,COUT,Isin
0,1.000000,MEDIUM,PRINCIPALE,APPARTEMENT,1.0,PROPRIO,JUR1,3500.0,NIVEAU_1,B40,...,0.0,0,5,2017,B,0,0,0.0,0.0,0.0
1,0.824657,CONFORT,PRINCIPALE,MAISON,NaN,PROPRIO,JUR1,0.0,NIVEAU_1,A11,...,0.0,0,9,2018,A,0,0,0.0,0.0,0.0
2,1.000000,ESSENTIEL,PRINCIPALE,APPARTEMENT,3.0,LOCATAIRE,JUR1,35000.0,NIVEAU_1,B32,...,0.0,0,11,2017,B,0,0,0.0,0.0,0.0
3,1.000000,ESSENTIEL,SECONDAIRE,MAISON,2.0,LOCATAIRE,JUR1,9000.0,NIVEAU_1,C24,...,0.0,0,13,2018,C,0,0,0.0,0.0,0.0
4,1.000000,ESSENTIEL,PRINCIPALE,MAISON,1.0,LOCATAIRE,JUR1,20000.0,NIVEAU_1,C9,...,0.0,0,14,2017,C,0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155646,0.680327,MEDIUM,PRINCIPALE,APPARTEMENT,2.0,PROPRIO,JUR1,3500.0,NIVEAU_1,A12,...,0.0,0,499986,2017,A,0,0,0.0,0.0,0.0
155647,1.000000,MEDIUM,PRINCIPALE,MAISON,2.0,PROPRIO,JUR1,9000.0,NIVEAU_1,B19,...,0.0,0,499989,2016,B,0,0,0.0,0.0,0.0
155648,0.569863,ALL_INCLUDE,PRINCIPALE,MAISON,1.0,PROPRIO,JUR1,3500.0,NIVEAU_1,A9,...,NaN,0,499991,2018,A,0,0,0.0,0.0,0.0
155649,0.597260,CONFORT,PRINCIPALE,MAISON,2.0,PROPRIO,JUR1,3500.0,NIVEAU_1,C23,...,0.0,0,499993,2016,C,0,0,0.0,0.0,0.0


In [58]:
random_state=42

On fait une fois pour toute le split entre train et val ici en faisant un stratified sampling sur l'indicatrice des sinistres du fait de sa faible représentation

In [115]:
df_train, df_val = train_test_split(df_merged, test_size=0.2, random_state=random_state, stratify=df_merged['Isin'])

In [116]:
y_train = df_train[['id','EXPO','NB','COUT', 'Isin']]
y_val = df_val[['id','EXPO','NB','COUT', 'Isin']]

In [117]:
X_train = df_train.drop(['NB','COUT', 'Isin'], axis=1)
X_val = df_val.drop(['NB','COUT', 'Isin'], axis=1)

In [118]:
X_test=expo_test

Les données sont stockées dans le répertoire `interim` pour un accès simplifié par les autres notebooks et avoir une source de vérité unique

In [119]:
# On exporte les données

#df
df_merged.to_parquet('/mnt/data/interim/df_merged.gzip',compression='gzip')
df_train.to_parquet('/mnt/data/interim/df_train.gzip',compression='gzip')
df_val.to_parquet('/mnt/data/interim/df_val.gzip',compression='gzip')


#X
X_train.to_parquet('/mnt/data/interim/X_train.gzip',compression='gzip')
X_val.to_parquet('/mnt/data/interim/X_val.gzip',compression='gzip')
X_test.to_parquet('/mnt/data/interim/X_test.gzip',compression='gzip')

#y
y_train.to_parquet('/mnt/data/interim/y_train.gzip',compression='gzip')
y_val.to_parquet('/mnt/data/interim/y_val.gzip',compression='gzip')


Pour mémoire : code implémenté dans [sklearn](https://scikit-learn.org/stable/auto_examples/linear_model/plot_tweedie_regression_insurance_claims.html#sphx-glr-auto-examples-linear-model-plot-tweedie-regression-insurance-claims-py) mais que nous n'avons finalement pas repris

In [74]:
# Insurances companies are interested in modeling the Pure Premium, that is
# the expected total claim amount per unit of exposure for each policyholder
# in their portfolio:
#df_merged["PurePremium"] = df_merged["COUT"] / df_merged["EXPO"]

# This can be indirectly approximated by a 2-step modeling: the product of the
# Frequency times the average claim amount per claim:
#df_merged["Frequency"] = df_merged["NB"] / df_merged["EXPO"]
#df_merged["AvgClaimAmount"] = df_merged["COUT"] / np.fmax(df_merged["NB"], 1)